# Regression models: lags, dummies and autoregression

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.linear_model import Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, rand

import warnings
warnings.filterwarnings("ignore")



## Load data

Two datasets will be loaded:

1. Dataset for the creation of train, validation and test splits with NaNs using expanding window.
2. Baselines results.

In [ ]:
# Load CSV data into a DataFrame
trainvalidtest = pd.read_csv('trainvalidtest_ar_v2.csv')

baselines = pd.read_csv('baseline_results_v2.csv')

pd.set_option('display.max_columns', None)

In [ ]:
trainvalidtest

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

## Create dictionaries with train and validation splits, and train and test splits respectively

### Train splits

Train split length for validation and test are different, as in case of test, train split will cover also validation split from the same fold.

In [ ]:
# Create train splits (separately as it does not include NaNs)
splits_train_val = {'train': [], 'val': []}

for idx in range(len(trainvalidtest) - 16, len(trainvalidtest), 1):
    train_data = trainvalidtest.loc[:idx, :]
    valid_data = trainvalidtest.loc[idx + 1: idx + 4, :]

    train_quarters = list(zip(train_data['Year'], train_data['Quarter']))
    valid_quarters = list(zip(valid_data['Year'], valid_data['Quarter']))

    print(f'TRAIN: {train_quarters}, VALIDATION: {valid_quarters}')

    splits_train_val['train'].append(train_data)
    splits_train_val['val'].append(valid_data)

    if idx + 8 >= len(trainvalidtest) - 1:
        break

TRAIN: [(2008, 1), (2008, 2), (2008, 3), (2008, 4), (2009, 1), (2009, 2), (2009, 3), (2009, 4), (2010, 1), (2010, 2), (2010, 3), (2010, 4), (2011, 1), (2011, 2), (2011, 3), (2011, 4), (2012, 1), (2012, 2), (2012, 3), (2012, 4), (2013, 1), (2013, 2), (2013, 3), (2013, 4), (2014, 1), (2014, 2), (2014, 3), (2014, 4), (2015, 1), (2015, 2), (2015, 3), (2015, 4), (2016, 1), (2016, 2), (2016, 3), (2016, 4), (2017, 1), (2017, 2), (2017, 3), (2017, 4), (2018, 1), (2018, 2), (2018, 3), (2018, 4), (2019, 1), (2019, 2), (2019, 3), (2019, 4), (2020, 1)], VALIDATION: [(2020, 2), (2020, 3), (2020, 4), (2021, 1)]
TRAIN: [(2008, 1), (2008, 2), (2008, 3), (2008, 4), (2009, 1), (2009, 2), (2009, 3), (2009, 4), (2010, 1), (2010, 2), (2010, 3), (2010, 4), (2011, 1), (2011, 2), (2011, 3), (2011, 4), (2012, 1), (2012, 2), (2012, 3), (2012, 4), (2013, 1), (2013, 2), (2013, 3), (2013, 4), (2014, 1), (2014, 2), (2014, 3), (2014, 4), (2015, 1), (2015, 2), (2015, 3), (2015, 4), (2016, 1), (2016, 2), (2016, 3), (2

In [ ]:
# Create train splits (separately as it does not include NaNs)
splits_train_test = {'train': [], 'test': []}

for idx in range(len(trainvalidtest) - 12, len(trainvalidtest), 1):
    train_data = trainvalidtest.loc[:idx, :]
    test_data = trainvalidtest.loc[idx + 1: idx + 4, :]

    train_quarters = list(zip(train_data['Year'], train_data['Quarter']))
    test_quarters = list(zip(test_data['Year'], test_data['Quarter']))

    print(f'TRAIN: {train_quarters}, TEST: {test_quarters}')

    splits_train_test['train'].append(train_data)
    splits_train_test['test'].append(test_data)

    if idx + 4 >= len(trainvalidtest) - 1:
        break

TRAIN: [(2008, 1), (2008, 2), (2008, 3), (2008, 4), (2009, 1), (2009, 2), (2009, 3), (2009, 4), (2010, 1), (2010, 2), (2010, 3), (2010, 4), (2011, 1), (2011, 2), (2011, 3), (2011, 4), (2012, 1), (2012, 2), (2012, 3), (2012, 4), (2013, 1), (2013, 2), (2013, 3), (2013, 4), (2014, 1), (2014, 2), (2014, 3), (2014, 4), (2015, 1), (2015, 2), (2015, 3), (2015, 4), (2016, 1), (2016, 2), (2016, 3), (2016, 4), (2017, 1), (2017, 2), (2017, 3), (2017, 4), (2018, 1), (2018, 2), (2018, 3), (2018, 4), (2019, 1), (2019, 2), (2019, 3), (2019, 4), (2020, 1), (2020, 2), (2020, 3), (2020, 4), (2021, 1)], TEST: [(2021, 2), (2021, 3), (2021, 4), (2022, 1)]
TRAIN: [(2008, 1), (2008, 2), (2008, 3), (2008, 4), (2009, 1), (2009, 2), (2009, 3), (2009, 4), (2010, 1), (2010, 2), (2010, 3), (2010, 4), (2011, 1), (2011, 2), (2011, 3), (2011, 4), (2012, 1), (2012, 2), (2012, 3), (2012, 4), (2013, 1), (2013, 2), (2013, 3), (2013, 4), (2014, 1), (2014, 2), (2014, 3), (2014, 4), (2015, 1), (2015, 2), (2015, 3), (2015, 4

### Validation and test splits with NaNs

In [ ]:
# New validation dictionary
val_nan = {'val_nan': []}

# Identify columns with 'lag_X' in their names
lag_columns = [col for col in trainvalidtest.columns if 'lag_' in col]

for idx in range(len(trainvalidtest) - 16, len(trainvalidtest), 1):
    valid_data_nan = trainvalidtest.loc[idx + 1: idx + 4, :].copy()  # Create a copy
    valid_quarters_nan = list(zip(valid_data_nan['Year'], valid_data_nan['Quarter']))

    print(f'VALIDATION: {valid_quarters_nan}')

    # Update validation data with NaNs in lag columns using boolean indexing
    for col in lag_columns:
        lag = int(col.split('_')[-1])  # Extract lag index from column name
        nan_rows = len(valid_data_nan) - lag
        if lag <= 3:
            valid_data_nan.loc[valid_data_nan.index[-nan_rows:], col] = np.nan
        else:
            valid_data_nan[col] = valid_data_nan[col]

    # Store data in the splits_nan dictionary
    val_nan['val_nan'].append(valid_data_nan)

    if idx + 8 >= len(trainvalidtest) - 1:
        break

VALIDATION: [(2020, 2), (2020, 3), (2020, 4), (2021, 1)]
VALIDATION: [(2020, 3), (2020, 4), (2021, 1), (2021, 2)]
VALIDATION: [(2020, 4), (2021, 1), (2021, 2), (2021, 3)]
VALIDATION: [(2021, 1), (2021, 2), (2021, 3), (2021, 4)]
VALIDATION: [(2021, 2), (2021, 3), (2021, 4), (2022, 1)]
VALIDATION: [(2021, 3), (2021, 4), (2022, 1), (2022, 2)]
VALIDATION: [(2021, 4), (2022, 1), (2022, 2), (2022, 3)]
VALIDATION: [(2022, 1), (2022, 2), (2022, 3), (2022, 4)]


In [ ]:
# New test dictionary
test_nan = {'test_nan': []}

# Identify columns with 'lag_X' in their names
lag_columns = [col for col in trainvalidtest.columns if 'lag_' in col]

for idx in range(len(trainvalidtest) - 16, len(trainvalidtest), 1):
    test_data_nan = trainvalidtest.loc[idx + 5: idx + 8, :].copy()
    test_quarters_nan = list(zip(test_data_nan['Year'], test_data_nan['Quarter']))

    print(f'TEST: {test_quarters_nan}')

    # Update test data with NaNs in lag columns using boolean indexing
    for col in lag_columns:
        lag = int(col.split('_')[-1])  # Extract lag index from column name
        nan_rows = len(test_data_nan) - lag
        if lag <= 3:
            test_data_nan.loc[test_data_nan.index[-nan_rows:], col] = np.nan
        else:
            test_data_nan[col] = test_data_nan[col]

    # Store data in the splits_nan dictionary
    test_nan['test_nan'].append(test_data_nan)

    if idx + 8 >= len(trainvalidtest) - 1:
        break

TEST: [(2021, 2), (2021, 3), (2021, 4), (2022, 1)]
TEST: [(2021, 3), (2021, 4), (2022, 1), (2022, 2)]
TEST: [(2021, 4), (2022, 1), (2022, 2), (2022, 3)]
TEST: [(2022, 1), (2022, 2), (2022, 3), (2022, 4)]
TEST: [(2022, 2), (2022, 3), (2022, 4), (2023, 1)]
TEST: [(2022, 3), (2022, 4), (2023, 1), (2023, 2)]
TEST: [(2022, 4), (2023, 1), (2023, 2), (2023, 3)]
TEST: [(2023, 1), (2023, 2), (2023, 3), (2023, 4)]


In [ ]:
# New dictionary combining train and val_nan
splits_val = {'train': splits_train_val['train'], 'val': val_nan['val_nan']}

In [ ]:
splits_val['train'][0].tail()

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

In [ ]:
splits_val['val'][0]

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

In [ ]:
# New dictionary combining train and test_nan
splits_test = {'train': splits_train_test['train'], 'test': test_nan['test_nan']}

In [ ]:
splits_test['train'][0].tail()

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

In [ ]:
splits_test['test'][0]

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

In [ ]:
# Calculate number of available attributes for each forecast horizon step
non_nan_counts = []

# Iterate over each row in the DataFrame
for index, row in splits_test['test'][0].iterrows():
    # Count non-NaN values in the current row and append to the list
    non_nan_counts.append(row.count())

# Convert the list to a Series
non_nan_counts_series = pd.Series(non_nan_counts, index=splits_test['test'][0].index)

print(non_nan_counts_series - 3) #subtract 'Year', 'Quarter' and 'NCN_sales_est'

53    293
54    257
55    221
56    185
dtype: int64


## Regression test with NaN values in validation and test

In [ ]:
regressors = {
    "Ridge": Ridge(),
    "Elastic Net": ElasticNet(),
    "Random Forest": RandomForestRegressor(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "XGBoost": XGBRegressor(),
}

### Validation

In [ ]:
# ITERATIONS WITH RANDOM SEEDS
reg_names_nan = []
cv_MAPE_nan = []
cv_R2_nan = []
cv_RMSE_nan = []
cv_folds_nan = []
all_preds = {}  # To store predictions for each regressor
preds_and_actuals = []

random_seeds = [42, 24, 99, 2024, 69, 1989, 2, 1972, 3, 16]

for seed in random_seeds:
    print(f"Random Seed: {seed}")
    f = 1

    for train, val in zip(splits_val['train'], splits_val['val']):
        print('Fold: ', f)

        X_train = train.drop(columns=['NCN_sales_est','Year', 'Quarter'], axis=1)
        y_train = train['NCN_sales_est']
        X_val = val.drop(columns=['NCN_sales_est','Year', 'Quarter'], axis=1)
        y_val = val['NCN_sales_est']

        sc = MinMaxScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_val = pd.DataFrame(sc.transform(X_val), columns=X_val.columns)

        for reg_name, reg in regressors.items():
            reg_names_nan.append(reg_name)

            y_val_pred = []

            for i in range(0, len(X_val)):
                X_val_test = X_val.iloc[i]
                X_val_test = X_val_test.to_frame().T
                nan_columns = X_val_test.columns[X_val_test.isna().any()].tolist()
                X_val_clean = X_val_test.drop(columns=nan_columns)
                X_train_clean = X_train.drop(columns=nan_columns)

                # Skip setting random seed for this model (not a parameter)
                if reg_name not in ['K-Nearest Neighbors']:
                    reg.set_params(random_state=seed)  # Set random seed

                reg.fit(X_train_clean, y_train)
                prediction = reg.predict(X_val_clean)
                y_val_pred.append(prediction)

            y_val_pred = [val[0] for val in y_val_pred]  # Flatten

            rmse = mean_squared_error(y_val, y_val_pred, squared=False)
            mape = mean_absolute_percentage_error(y_val, y_val_pred)
            r2 = r2_score(y_val, y_val_pred)
            cv_MAPE_nan.append(mape)
            cv_R2_nan.append(r2)
            cv_RMSE_nan.append(rmse)
            cv_folds_nan.append(i)

            if reg_name not in all_preds:
                all_preds[reg_name] = []
            all_preds[reg_name].append(np.array(y_val_pred))

            preds_and_actuals.append({
                'reg_name': reg_name,
                'train_preds': np.array(reg.predict(X_train_clean)),
                'val_preds': np.array(y_val_pred),
                'train_actuals': np.array(y_train),
                'val_actuals': np.array(y_val)
            })

            print(reg_name)
            print('RMSE:', rmse)
            print('MAPE:', mape)
            print('R2:', r2)

        f += 1
        print('###')
        print()

# Convert the lists of predictions to numpy arrays for each regressor
for reg_name, preds_list in all_preds.items():
    all_preds[reg_name] = np.array(preds_list)


# Convert the list of predictions to a numpy array
preds_and_actuals = np.array(preds_and_actuals)

Random Seed: 42
Fold:  1
Ridge
RMSE: 6511.022443025221
MAPE: 0.0787959907789991
R2: 0.684239014170353
Elastic Net
RMSE: 9448.636233446532
MAPE: 0.11350114356092367
R2: 0.33503567978888993
Random Forest
RMSE: 10775.565716134119
MAPE: 0.11474433091118126
R2: 0.13515108615929117
K-Nearest Neighbors
RMSE: 10305.755831087792
MAPE: 0.12889309805536187
R2: 0.20892113439393145
XGBoost
RMSE: 8527.701857104363
MAPE: 0.12171049263665037
R2: 0.4583433228501751
###

Fold:  2
Ridge
RMSE: 5173.236062068044
MAPE: 0.0784503050687834
R2: 0.7268056354964542
Elastic Net
RMSE: 8671.769327346743
MAPE: 0.11779400174598098
R2: 0.2323511960198048
Random Forest
RMSE: 15154.12907392817
MAPE: 0.16982411904631595
R2: -1.344277977413907
K-Nearest Neighbors
RMSE: 5861.618585339719
MAPE: 0.09342737283259579
R2: 0.649262457393054
XGBoost
RMSE: 16793.28337724029
MAPE: 0.21498348068720893
R2: -1.8788456240364382
###

Fold:  3
Ridge
RMSE: 6188.580709854844
MAPE: 0.0899838653173774
R2: 0.8279627713852392
Elastic Net
RMSE:

In [ ]:
# Calculate MAPE for each fold
mape_per_fold = []

for fold_data in preds_and_actuals:
    reg_name = fold_data['reg_name']
    train_preds = fold_data['train_preds']
    val_preds = fold_data['val_preds']
    train_actuals = fold_data['train_actuals']
    val_actuals = fold_data['val_actuals']

    train_mape = mean_absolute_percentage_error(train_actuals, train_preds)
    val_mape = mean_absolute_percentage_error(val_actuals, val_preds)

    mape_per_fold.append({
        'reg_name': reg_name,
        'train_mape': train_mape,
        'val_mape': val_mape
    })

# Display MAPE for each fold
for fold_data in mape_per_fold:
    print(f"Regressor: {fold_data['reg_name']}")
    print(f"Train MAPE: {fold_data['train_mape']}")
    print(f"Validation MAPE: {fold_data['val_mape']}")
    print("###")

Regressor: Ridge
Train MAPE: 0.05574999292123823
Validation MAPE: 0.0787959907789991
###
Regressor: Elastic Net
Train MAPE: 0.15150235132121054
Validation MAPE: 0.11350114356092367
###
Regressor: Random Forest
Train MAPE: 0.07375346408692625
Validation MAPE: 0.11474433091118126
###
Regressor: K-Nearest Neighbors
Train MAPE: 0.16729427070100067
Validation MAPE: 0.12889309805536187
###
Regressor: XGBoost
Train MAPE: 7.198723520677984e-08
Validation MAPE: 0.12171049263665037
###
Regressor: Ridge
Train MAPE: 0.055624729191632874
Validation MAPE: 0.0784503050687834
###
Regressor: Elastic Net
Train MAPE: 0.15169193465179084
Validation MAPE: 0.11779400174598098
###
Regressor: Random Forest
Train MAPE: 0.0754630230771723
Validation MAPE: 0.16982411904631595
###
Regressor: K-Nearest Neighbors
Train MAPE: 0.16452948171462986
Validation MAPE: 0.09342737283259579
###
Regressor: XGBoost
Train MAPE: 7.424912864819501e-08
Validation MAPE: 0.21498348068720893
###
Regressor: Ridge
Train MAPE: 0.0546956

In [ ]:
# Initialize dictionaries to store mean MAPEs for train and validation sets
mean_train_mape_per_reg = {}
mean_val_mape_per_reg = {}

# Loop through each regressor
for reg_name in regressors.keys():
    # Initialize lists to store MAPEs for each fold
    train_mapes = []
    val_mapes = []

    # Loop through each fold data
    for fold_data in mape_per_fold:
        if fold_data['reg_name'] == reg_name:
            train_mapes.append(fold_data['train_mape'])
            val_mapes.append(fold_data['val_mape'])

    # Calculate mean MAPEs for train and validation sets
    mean_train_mape = np.mean(train_mapes)
    mean_val_mape = np.mean(val_mapes)

    # Store mean MAPEs in dictionaries
    mean_train_mape_per_reg[reg_name] = mean_train_mape
    mean_val_mape_per_reg[reg_name] = mean_val_mape

# Display mean MAPEs for train and validation sets
for reg_name in regressors.keys():
    print(f"Regressor: {reg_name}")
    print(f"Mean Train MAPE: {mean_train_mape_per_reg[reg_name]}")
    print(f"Mean Validation MAPE: {mean_val_mape_per_reg[reg_name]}")
    print("###")


Regressor: Ridge
Mean Train MAPE: 0.05495962930487166
Mean Validation MAPE: 0.058393196156307316
###
Regressor: Elastic Net
Mean Train MAPE: 0.15276239342222678
Mean Validation MAPE: 0.14859821807385598
###
Regressor: Random Forest
Mean Train MAPE: 0.07194397587350429
Mean Validation MAPE: 0.1539173947900482
###
Regressor: K-Nearest Neighbors
Mean Train MAPE: 0.16098642006616684
Mean Validation MAPE: 0.1642259327390098
###
Regressor: XGBoost
Mean Train MAPE: 7.170221795029363e-08
Mean Validation MAPE: 0.17235031525747443
###


In [ ]:
# Initialize a dictionary to store mean MAPEs for train and validation sets
reg_results_data = {'Regressor': [], 'Mean Train MAPE': [], 'Mean Validation MAPE': []}

# Loop through each regressor
for reg_name in regressors.keys():
    # Initialize lists to store MAPEs for each fold
    train_mapes = []
    val_mapes = []

    # Loop through each fold data
    for fold_data in mape_per_fold:
        if fold_data['reg_name'] == reg_name:
            train_mapes.append(fold_data['train_mape'])
            val_mapes.append(fold_data['val_mape'])

    # Calculate mean MAPEs for train and validation sets
    mean_train_mape = np.mean(train_mapes)
    mean_val_mape = np.mean(val_mapes)

    # Store results in the dictionary
    reg_results_data['Regressor'].append(reg_name)
    reg_results_data['Mean Train MAPE'].append(round(mean_train_mape*100, 2))
    reg_results_data['Mean Validation MAPE'].append(round(mean_val_mape*100, 2))

# Create a DataFrame from the dictionary
reg_results = pd.DataFrame(reg_results_data)

# Display the DataFrame
reg_results


,Regressor,Mean Train MAPE,Mean Validation MAPE
0,Ridge,5.50,5.84
1,Elastic Net,15.28,14.86
2,Random Forest,7.19,15.39
3,K-Nearest Neighbors,16.10,16.42
4,XGBoost,0.00,17.24


In [ ]:
reg_results_sort = reg_results.sort_values(by=('Mean Validation MAPE'), ascending=True)
reg_results_sort

,Regressor,Mean Train MAPE,Mean Validation MAPE
0,Ridge,5.50,5.84
1,Elastic Net,15.28,14.86
2,Random Forest,7.19,15.39
3,K-Nearest Neighbors,16.10,16.42
4,XGBoost,0.00,17.24


In [ ]:
# Dataframe for all error metrics
df_results_nan = pd.DataFrame()
df_results_nan['fold'] = cv_folds_nan
df_results_nan['regressor'] = reg_names_nan
df_results_nan['RMSE'] = cv_RMSE_nan
df_results_nan['MAPE'] = cv_MAPE_nan
df_results_nan['R2'] = cv_R2_nan

In [ ]:
df_results_agg_nan = df_results_nan.groupby('regressor').agg({'RMSE': ['mean'], 'MAPE': ['mean'], 'R2': ['mean']})
df_results_agg_nan = df_results_agg_nan.sort_values(by=('MAPE', 'mean'), ascending=True)
df_results_agg_nan

,RMSE,MAPE,R2
,mean,mean,mean
regressor,,,
Ridge,4780.021936,0.058393,0.744110
Elastic Net,12109.378950,0.148598,-0.794010
Random Forest,13806.725537,0.153917,-1.492176
K-Nearest Neighbors,12807.132048,0.164226,-1.519523
XGBoost,13594.553963,0.172350,-1.576266


### Test

In [ ]:
# ITERATIONS WITH RANDOM SEEDS
reg_names_nan = []
test_MAPE_nan = []
test_R2_nan = []
test_RMSE_nan = []
test_folds_nan = []
all_preds_test = {}  # To store predictions for each regressor
preds_and_actuals_test = []

random_seeds = [42, 24, 99, 2024, 69, 1989, 2, 1972, 3, 16]

for seed in random_seeds:
    print(f"Random Seed: {seed}")
    f = 1

    for train, test in zip(splits_test['train'], splits_test['test']):
        print('Fold: ', f)

        X_train = train.drop(columns=['NCN_sales_est','Year', 'Quarter'], axis=1)
        y_train = train['NCN_sales_est']
        X_test = test.drop(columns=['NCN_sales_est','Year', 'Quarter'], axis=1)
        y_test = test['NCN_sales_est']

        sc = MinMaxScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = pd.DataFrame(sc.transform(X_test), columns=X_test.columns)

        for reg_name, reg in regressors.items():
            reg_names_nan.append(reg_name)

            y_test_pred = []

            for i in range(0, len(X_test)):
                X_test_test = X_test.iloc[i]
                X_test_test = X_test_test.to_frame().T
                nan_columns = X_test_test.columns[X_test_test.isna().any()].tolist()
                X_test_clean = X_test_test.drop(columns=nan_columns)
                X_train_clean = X_train.drop(columns=nan_columns)

                # Skip setting random seed for this model
                if reg_name not in ['K-Nearest Neighbors']:
                    reg.set_params(random_state=seed)  # Set random seed

                reg.fit(X_train_clean, y_train)
                prediction = reg.predict(X_test_clean)
                y_test_pred.append(prediction)

            y_test_pred = [test[0] for test in y_test_pred]  # Flatten

            rmse = mean_squared_error(y_test, y_test_pred, squared=False)
            mape = mean_absolute_percentage_error(y_test, y_test_pred)
            r2 = r2_score(y_test, y_test_pred)
            test_MAPE_nan.append(mape)
            test_R2_nan.append(r2)
            test_RMSE_nan.append(rmse)
            test_folds_nan.append(i)

            if reg_name not in all_preds_test:
                all_preds_test[reg_name] = []
            all_preds_test[reg_name].append(np.array(y_test_pred))

            preds_and_actuals_test.append({
                'reg_name': reg_name,
                'train_preds': np.array(reg.predict(X_train_clean)),
                'test_preds': np.array(y_test_pred),
                'train_actuals': np.array(y_train),
                'test_actuals': np.array(y_test)
            })

            print(reg_name)
            print('RMSE:', rmse)
            print('MAPE:', mape)
            print('R2:', r2)

        f += 1
        print('###')
        print()

# Convert the lists of predictions to numpy arrays for each regressor
for reg_name, preds_list in all_preds_test.items():
    all_preds_test[reg_name] = np.array(preds_list)


# Convert the list of predictions to a numpy array
preds_and_actuals_test = np.array(preds_and_actuals_test)

Random Seed: 42
Fold:  1
Ridge
RMSE: 5914.078764651401
MAPE: 0.06390820477782191
R2: 0.458033076208026
Elastic Net
RMSE: 15894.429142723633
MAPE: 0.20949905471259012
R2: -2.9146094766199475
Random Forest
RMSE: 16838.21507638354
MAPE: 0.20195888414051028
R2: -3.393298164098142
K-Nearest Neighbors
RMSE: 17075.45243792972
MAPE: 0.22742483968498794
R2: -3.5179665731654586
XGBoost
RMSE: 16267.737364471763
MAPE: 0.18212524633857824
R2: -3.10065165877202
###

Fold:  2
Ridge
RMSE: 2605.8798325634266
MAPE: 0.03365260507537943
R2: 0.8763153285656118
Elastic Net
RMSE: 14651.821460707943
MAPE: 0.18931934340039597
R2: -2.9101223415520985
Random Forest
RMSE: 16942.046859516417
MAPE: 0.20570374783273696
R2: -4.228039585869501
K-Nearest Neighbors
RMSE: 18375.346031843867
MAPE: 0.24379119722839765
R2: -5.150043143723954
XGBoost
RMSE: 15124.872242038477
MAPE: 0.1979515140945559
R2: -3.166683759901468
###

Fold:  3
Ridge
RMSE: 2487.5835739096437
MAPE: 0.02358680753399451
R2: 0.8910440921724191
Elastic Ne

In [ ]:
# Calculate MAPE for each fold
mape_per_fold_test = []

for fold_data in preds_and_actuals_test:
    reg_name = fold_data['reg_name']
    train_preds = fold_data['train_preds']
    test_preds = fold_data['test_preds']
    train_actuals = fold_data['train_actuals']
    test_actuals = fold_data['test_actuals']

    train_mape = mean_absolute_percentage_error(train_actuals, train_preds)
    test_mape = mean_absolute_percentage_error(test_actuals, test_preds)

    mape_per_fold_test.append({
        'reg_name': reg_name,
        'train_mape': train_mape,
        'test_mape': test_mape
    })

# Display MAPE for each fold
for fold_data in mape_per_fold_test:
    print(f"Regressor: {fold_data['reg_name']}")
    print(f"Train MAPE: {fold_data['train_mape']}")
    print(f"Test MAPE: {fold_data['test_mape']}")
    print("###")

Regressor: Ridge
Train MAPE: 0.055605802835355336
Test MAPE: 0.06390820477782191
###
Regressor: Elastic Net
Train MAPE: 0.15010548699806145
Test MAPE: 0.20949905471259012
###
Regressor: Random Forest
Train MAPE: 0.07391977381129979
Test MAPE: 0.20195888414051028
###
Regressor: K-Nearest Neighbors
Train MAPE: 0.1577666190015726
Test MAPE: 0.22742483968498794
###
Regressor: XGBoost
Train MAPE: 6.615298308638667e-08
Test MAPE: 0.18212524633857824
###
Regressor: Ridge
Train MAPE: 0.055496006202434674
Test MAPE: 0.03365260507537943
###
Regressor: Elastic Net
Train MAPE: 0.1541614504776757
Test MAPE: 0.18931934340039597
###
Regressor: Random Forest
Train MAPE: 0.07095576990834164
Test MAPE: 0.20570374783273696
###
Regressor: K-Nearest Neighbors
Train MAPE: 0.1581755626955348
Test MAPE: 0.24379119722839765
###
Regressor: XGBoost
Train MAPE: 6.50102008599842e-08
Test MAPE: 0.1979515140945559
###
Regressor: Ridge
Train MAPE: 0.05356129956567891
Test MAPE: 0.02358680753399451
###
Regressor: Elas

In [ ]:
# Initialize dictionaries to store mean MAPEs for train and test sets
mean_train_mape_per_reg = {}
mean_test_mape_per_reg = {}

# Loop through each regressor
for reg_name in regressors.keys():
    # Initialize lists to store MAPEs for each fold
    train_mapes = []
    test_mapes = []

    # Loop through each fold data
    for fold_data in mape_per_fold_test:
        if fold_data['reg_name'] == reg_name:
            train_mapes.append(fold_data['train_mape'])
            test_mapes.append(fold_data['test_mape'])

    # Calculate mean MAPEs for train and test sets
    mean_train_mape = np.mean(train_mapes)
    mean_test_mape = np.mean(test_mapes)

    # Store mean MAPEs in dictionaries
    mean_train_mape_per_reg[reg_name] = mean_train_mape
    mean_test_mape_per_reg[reg_name] = mean_test_mape

# Display mean MAPEs for train and test sets
for reg_name in regressors.keys():
    print(f"Regressor: {reg_name}")
    print(f"Mean Train MAPE: {mean_train_mape_per_reg[reg_name]}")
    print(f"Mean Test MAPE: {mean_test_mape_per_reg[reg_name]}")
    print("###")

Regressor: Ridge
Mean Train MAPE: 0.054096943345458726
Mean Test MAPE: 0.08958994275166202
###
Regressor: Elastic Net
Mean Train MAPE: 0.1589985931988386
Mean Test MAPE: 0.14377738319960487
###
Regressor: Random Forest
Mean Train MAPE: 0.07031893978048716
Mean Test MAPE: 0.19051333730269232
###
Regressor: K-Nearest Neighbors
Mean Train MAPE: 0.1579659305968789
Mean Test MAPE: 0.18382627637770946
###
Regressor: XGBoost
Mean Train MAPE: 7.44204480772955e-08
Mean Test MAPE: 0.18378702299780497
###


In [ ]:
# Initialize a dictionary to store mean MAPEs for train and test sets
reg_results_data_test = {'Regressor': [], 'Mean Train MAPE': [], 'Mean Test MAPE': []}

# Loop through each regressor
for reg_name in regressors.keys():
    # Initialize lists to store MAPEs for each fold
    train_mapes = []
    test_mapes = []

    # Loop through each fold data
    for fold_data in mape_per_fold_test:
        if fold_data['reg_name'] == reg_name:
            train_mapes.append(fold_data['train_mape'])
            test_mapes.append(fold_data['test_mape'])

    # Calculate mean MAPEs for train and test sets
    mean_train_mape = np.mean(train_mapes)
    mean_test_mape = np.mean(test_mapes)

    # Store results in the dictionary
    reg_results_data_test['Regressor'].append(reg_name)
    reg_results_data_test['Mean Train MAPE'].append(round(mean_train_mape*100, 2))
    reg_results_data_test['Mean Test MAPE'].append(round(mean_test_mape*100, 2))

# Create a DataFrame from the dictionary
reg_results_test = pd.DataFrame(reg_results_data_test)

# Display the DataFrame
reg_results_test

,Regressor,Mean Train MAPE,Mean Test MAPE
0,Ridge,5.41,8.96
1,Elastic Net,15.90,14.38
2,Random Forest,7.03,19.05
3,K-Nearest Neighbors,15.80,18.38
4,XGBoost,0.00,18.38


### Results

In [ ]:
reg_results_besttest = reg_results.merge(reg_results_test[['Regressor', 'Mean Test MAPE']], on='Regressor', how='left')
reg_results_besttest = reg_results_besttest.sort_values(by=('Mean Test MAPE'), ascending=True)
reg_results_besttest = reg_results_besttest.reset_index(drop=True)
reg_results_besttest

,Regressor,Mean Train MAPE,Mean Validation MAPE,Mean Test MAPE
0,Ridge,5.50,5.84,8.96
1,Elastic Net,15.28,14.86,14.38
2,K-Nearest Neighbors,16.10,16.42,18.38
3,XGBoost,0.00,17.24,18.38
4,Random Forest,7.19,15.39,19.05


# HPO

In [ ]:
random_state = 0

In [ ]:
# Labels of regression models
names = [
    "Ridge",
    "Elastic Net",
    "Random Forest",
    "K-Nearest Neighbors",
    "XGBoost"
]

# Default/baseline regression model classes
regression_models = [
    Ridge(),
    ElasticNet(),
    RandomForestRegressor(),
    KNeighborsRegressor(),
    XGBRegressor()
]

In [ ]:
# Define individual search spaces manually
regression_search_spaces = [
    # Ridge
    {
        'alpha': hp.uniform('alpha', 0.01, 200.0),
        'fit_intercept': hp.choice('fit_intercept', [True, False]),
        'solver': hp.choice('solver', ['auto']),
        'max_iter': hp.choice('max_iter', range(100, 2000, 10)),
        'tol': hp.uniform('tol', 1e-6, 1e-3),
        'random_state': random_state
    },

    # Elastic Net
    {
       'alpha': hp.uniform('alpha', 0.01, 200.0),
       'copy_X': hp.choice('copy_X', [True, False]),
       'l1_ratio': hp.uniform('l1_ratio', 0, 1),
       'fit_intercept': hp.choice('fit_intercept', [True, False]),
       'max_iter': hp.choice('max_iter', range(100, 2000, 10)),
       'tol': hp.uniform('tol', 1e-6, 1e-3),
       'random_state': random_state
   },

    # Random Forest
    {
        'max_depth': hp.choice('max_depth', range(1, 40)),
        'min_samples_split': hp.uniform('min_samples_split', 0.0, 1.0),
        'max_leaf_nodes': hp.choice('max_leaf_nodes', range(2, 50)),
        'n_estimators': hp.choice('n_estimators', range(50, 250, 1)),
        'random_state': random_state
    },

    # K-Nearest Neighbors
    {
        'n_neighbors': hp.choice('n_neighbors', range(2, 20, 1)),
        'p': hp.choice('p', [1, 5]),
        'weights': hp.choice('weights', ['uniform', 'distance']),
        'algorithm': hp.choice('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),
        'leaf_size': hp.choice('leaf_size', range(10, 40, 1)),
        'metric': hp.choice('metric', ['minkowski', 'euclidean', 'manhattan'])
    },

    # XGBoost
    {
        'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
        'reg_lambda': hp.uniform('reg_lambda', 0.01, 1),
        'reg_alpha': hp.uniform('reg_alpha', 0.01, 1),
        'updater': hp.choice('updater', ['shotgun', 'coord_descent']),
        'feature_selector': hp.choice('feature_selector', ['cyclic', 'shuffle']),
        'booster': hp.choice('booster', ['gblinear']),
        'random_state': random_state
    }

]



### Loop

In [ ]:
ITERATIONS = 300

In [ ]:
# Initialize a dataframe for results collection
hpo_results = pd.DataFrame(columns = ['regressor_name', 'HPO train MAPE', 'Train std', 'HPO valid MAPE',
                                      'Valid std', 'HPO test MAPE', 'Test std', 'runtime_hpo', 'best_params',
                                      'FH_MAPEs', 'train_MAPE_per_split', 'cv_MAPE_per_split', 'test_MAPE_per_split'])

# Define the best baseline score
best_baseline_score = reg_results_besttest.loc[0, 'Mean Test MAPE']

# Define how many iterations should be done
n_trials = ITERATIONS
SEED = 1972
# All individual trials will be saved here
trials_dict = {}

# Start the timer to measure the runtime of the entire pipeline
hpo_time_start = time.time()

# Run the HPO and get the best params for each regressor
for i in range(len(reg_results_besttest)):

    # To improve the readability of code, creating the following objects:
    regressor_name = reg_results_besttest.loc[i, 'Regressor']
    regressor_class = regression_models[names.index(regressor_name)] # fetch from the list not df for baseline
    regressor_search_space = regression_search_spaces[names.index(regressor_name)]

    # To improve the readability of output:
    print()
    print('----------------------------------------------------------------------')
    print(f'Using *{regressor_name}* for estimation.')
    print(regressor_search_space)
    print('----------------------------------------------------------------------')

    # A objective function for receiving the CV and train scores for each model
    def hyperopt_cv_score(params):
        cv_MAPE_nan = []
        train_MAPE_nan = []

        if 'n_estimators' in params:
            # Convert 'n_estimators' to an integer
            params['n_estimators'] = int(params['n_estimators'])

        for train, val in zip(splits_val['train'], splits_val['val']):
            X_train = train.drop(columns=['NCN_sales_est', 'Year', 'Quarter'], axis=1)
            y_train = train['NCN_sales_est']
            X_val = val.drop(columns=['NCN_sales_est', 'Year', 'Quarter'], axis=1)
            y_val = val['NCN_sales_est']
            sc = MinMaxScaler()
            X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
            X_val = pd.DataFrame(sc.transform(X_val), columns=X_val.columns)

            y_val_pred = []

            model = regressor_class.set_params(**params)  # use the regressor from the list

            for i in range(0, len(X_val)):
                X_val_test = X_val.iloc[i]
                X_val_test = X_val_test.to_frame().T
                nan_columns = X_val_test.columns[X_val_test.isna().any()].tolist()
                X_val_clean = X_val_test.drop(columns=nan_columns)
                X_train_clean = X_train.drop(columns=nan_columns)
                model.fit(X_train_clean, y_train)
                prediction = model.predict(X_val_clean)
                y_val_pred.append(prediction)

            y_val_pred = [val[0] for val in y_val_pred]  # Flatten

            mape = mean_absolute_percentage_error(y_val, y_val_pred)

            cv_MAPE_nan.append(mape)

            # Calculate MAPE on the training set
            y_train_pred = model.predict(X_train_clean)
            train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
            train_MAPE_nan.append(train_mape)

        mean_mape = np.mean(cv_MAPE_nan)
        mean_train_mape = np.mean(train_MAPE_nan)

        return mean_mape, mean_train_mape, cv_MAPE_nan, train_MAPE_nan

    # A helper function for finding the best model
    def f(params):
        global best_cv_score
        global best_train_score
        global best_params
        global best_time
        global best_cv_MAPEs
        global best_train_MAPEs

        cv_score, train_score, cv_MAPE_nan, train_MAPE_nan = hyperopt_cv_score(params)

        if cv_score < best_cv_score:
          best_cv_score = cv_score
          best_train_score = train_score
          best_params = params
          best_time = round(time.time() - start_time, 4)  # track how much time it took to find the best params
          # Store the MAPEs for the best model
          best_cv_MAPEs = cv_MAPE_nan
          best_train_MAPEs = train_MAPE_nan

          print(f'Better CV score: {best_cv_score}')
          print(f'Better train score: {best_train_score}')
          print(f'Parameter combination: {best_params}')
          print(f'Time until beating the baseline: {best_time}s')

        return {'loss': cv_score,
                'status': STATUS_OK}

    # Defining global variables to be updated
    best_cv_score = float('inf')
    best_train_score = float('inf')
    best_params = None # best hyperparameter combination
    best_time = 0 # runtime until the best CV score is computed
    best_cv_MAPEs = []  # Added to store the best CV MAPE
    best_train_MAPEs = []  # Added to store the best training MAPE

    trials = Trials() # store info at each step

    # Start running the algorithm and track time
    start_time = time.time()
    ## Hyperopt function
    best = fmin(f,regressor_search_space, # use the search space associated with the regressor
                algo = tpe.suggest,
                max_evals = n_trials, # how many evaluations?
                trials = trials,
                rstate=np.random.default_rng(SEED))
    # Save all trials
    trials_dict[regressor_name] = trials # save into regressor-trials
    print()
    print('######################################################################')
    # Print the summary of the best results
    print(f'Best training score in {n_trials} iterations: {best_train_score}.')
    print(f'Best validation score in {n_trials} iterations: {best_cv_score} ({best_time}s until found).')


    # Initialize an empty list to store test MAPE for each fold
    test_mape_list = []
    fh_mapes = []

    # Iterate through each fold in splits_test
    for fold_train, fold_test in zip(splits_test['train'], splits_test['test']):
        X_train_fold = fold_train.drop(columns=['NCN_sales_est','Year', 'Quarter'], axis=1)
        y_train_fold = fold_train['NCN_sales_est']
        X_test_fold = fold_test.drop(columns=['NCN_sales_est','Year', 'Quarter'], axis=1)
        y_test_fold = fold_test['NCN_sales_est']

        # Scale to [0,1] range using the same scaler as in training
        X_train_fold_scaled = pd.DataFrame(sc.transform(X_train_fold), columns=X_train_fold.columns)
        X_test_fold_scaled = pd.DataFrame(sc.transform(X_test_fold), columns=X_test_fold.columns)

        # Make predictions for the current fold
        y_test_pred_fold = []
        for i in range(len(X_test_fold_scaled)):
            X_test_nan_fold = X_test_fold_scaled.iloc[i]
            X_test_nan_fold = X_test_nan_fold.to_frame().T
            nan_columns_fold = X_test_nan_fold.columns[X_test_nan_fold.isna().any()].tolist()
            X_test_clean_fold = X_test_nan_fold.drop(columns=nan_columns_fold)
            X_train_clean_fold = X_train_fold_scaled.drop(columns=nan_columns_fold)
            regressor_instance = regressor_class.set_params(**best_params)
            regressor_instance.fit(X_train_clean_fold, y_train_fold)
            prediction_fold = regressor_instance.predict(X_test_clean_fold)
            y_test_pred_fold.append(prediction_fold[0])

        # Calculate MAPE for the current fold
        test_mape_fold = mean_absolute_percentage_error(y_test_fold, y_test_pred_fold)
        test_mape_list.append(test_mape_fold)
        # Initialize an empty list to store the absolute percentage errors
        apes_split = []

        # Iterate over each pair of predicted and actual values
        for pred, act in zip(y_test_pred_fold, y_test_fold):
            # Calculate the absolute percentage error
            abs_percent_error = abs((pred - act) / act)
            # Append the absolute percentage error to the list
            apes_split.append(abs_percent_error)
        fh_mapes.append(apes_split)

    # Calculate the mean MAPE across all folds
    test_mape_mean = np.mean(test_mape_list)
    test_mape_std = np.std(test_mape_list)
    # Calculate mean value for each member of the lists
    mean_values = [np.mean(member) for member in zip(*fh_mapes)]

    print(f'{regressor_name} MAPE for test data across all folds: {test_mape_mean}.')

    # Append the best results to the df
    hpo_results = hpo_results.append({'regressor_name': regressor_name,
                             'HPO train MAPE': best_train_score,
                             'Train std': np.std(best_train_MAPEs),
                             'HPO valid MAPE': best_cv_score,
                             'Valid std': np.std(best_cv_MAPEs),
                             'HPO test MAPE': test_mape_mean,
                             'Test std': test_mape_std,
                             'runtime_hpo': best_time,
                             'best_params': best_params,
                             'FH_MAPEs': mean_values,
                             'train_MAPE_per_split': best_train_MAPEs,
                             'cv_MAPE_per_split': best_cv_MAPEs,
                             'test_MAPE_per_split': test_mape_list}, ignore_index = True)
# Mark the end of the entire pipeline
hpo_time_end = time.time()
print()
print('######################################################################')
print(f'The duration of the entire HPO pipeline for {len(reg_results_besttest)} regressors across {n_trials} trials each: ')
print(f'{round(hpo_time_end - hpo_time_start, 5)} seconds')


----------------------------------------------------------------------
Using *Ridge* for estimation.
{'alpha': <hyperopt.pyll.base.Apply object at 0x7eabec3d86d0>, 'fit_intercept': <hyperopt.pyll.base.Apply object at 0x7eabec3db010>, 'solver': <hyperopt.pyll.base.Apply object at 0x7eabec3d8790>, 'max_iter': <hyperopt.pyll.base.Apply object at 0x7eabec3daef0>, 'tol': <hyperopt.pyll.base.Apply object at 0x7eabec3da080>, 'random_state': 0}
----------------------------------------------------------------------
Better CV score: 0.055474842830418786
Better train score: 0.042945627215358294
Parameter combination: {'alpha': 0.5457579345138338, 'fit_intercept': True, 'max_iter': 500, 'random_state': 0, 'solver': 'auto', 'tol': 0.0009659227193673929}
Time until beating the baseline: 0.6761s
Better CV score: 0.05547261000435553
Better train score: 0.042912587247484105
Parameter combination: {'alpha': 0.5448414388501107, 'fit_intercept': True, 'max_iter': 500, 'random_state': 0, 'solver': 'auto',

In [ ]:
hpo_results

,regressor_name,HPO train MAPE,Train std,HPO valid MAPE,Valid std,HPO test MAPE,Test std,runtime_hpo,best_params,FH_MAPEs,train_MAPE_per_split,cv_MAPE_per_split,test_MAPE_per_split
0,Ridge,0.042876,0.000482,0.055470,0.024359,0.098656,0.056013,92.9519,"{'alpha': 0.5438252426248501, 'fit_intercept':...","[0.08261525279811104, 0.06980042020881116, 0.1...","[0.04317532704468448, 0.042872883135561674, 0....","[0.09653562498312972, 0.06985888342346794, 0.0...","[0.07654920307129107, 0.04588245032601229, 0.0..."
1,Elastic Net,0.033444,0.001317,0.055751,0.026746,0.105524,0.052839,254.9202,"{'alpha': 9.973108826045255, 'copy_X': True, '...","[0.08847214244434784, 0.07839945934019509, 0.1...","[0.03198652928251123, 0.032189130781534146, 0....","[0.11724506861407157, 0.07023870950289694, 0.0...","[0.09367995606956717, 0.07080361652676895, 0.0..."
2,K-Nearest Neighbors,0.000000,0.000000,0.148266,0.047779,0.186641,0.043100,1.0994,"{'algorithm': 'brute', 'leaf_size': 39, 'metri...","[0.18547693791215658, 0.22947100158692083, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.12798394574174418, 0.05815775988032873, 0.1...","[0.20924416332038512, 0.2362024093676182, 0.15..."
3,XGBoost,0.067642,0.001235,0.057717,0.021651,0.086134,0.045209,717.1597,"{'booster': 'gblinear', 'feature_selector': 'c...","[0.07883665244247692, 0.06773080034489458, 0.0...","[0.06743986537476192, 0.06881291584713467, 0.0...","[0.0703990174229799, 0.06667889811138855, 0.09...","[0.06114039102739344, 0.01854096454474325, 0.0..."
4,Random Forest,0.076490,0.002545,0.146966,0.028906,0.183533,0.031418,556.6586,"{'max_depth': 38, 'max_leaf_nodes': 21, 'min_s...","[0.19007065800038656, 0.20236809026495145, 0.1...","[0.07942972944919592, 0.0805806648357526, 0.07...","[0.10649519204948915, 0.12835186644190952, 0.1...","[0.17870582493116696, 0.19241889064618928, 0.1..."


In [ ]:
# Sort the model results by test set MAPE
hpo_results = hpo_results.sort_values('HPO test MAPE', ascending=True).reset_index(drop=True)
hpo_results

,regressor_name,HPO train MAPE,Train std,HPO valid MAPE,Valid std,HPO test MAPE,Test std,runtime_hpo,best_params,FH_MAPEs,train_MAPE_per_split,cv_MAPE_per_split,test_MAPE_per_split
0,XGBoost,0.067642,0.001235,0.057717,0.021651,0.086134,0.045209,717.1597,"{'booster': 'gblinear', 'feature_selector': 'c...","[0.07883665244247692, 0.06773080034489458, 0.0...","[0.06743986537476192, 0.06881291584713467, 0.0...","[0.0703990174229799, 0.06667889811138855, 0.09...","[0.06114039102739344, 0.01854096454474325, 0.0..."
1,Ridge,0.042876,0.000482,0.055470,0.024359,0.098656,0.056013,92.9519,"{'alpha': 0.5438252426248501, 'fit_intercept':...","[0.08261525279811104, 0.06980042020881116, 0.1...","[0.04317532704468448, 0.042872883135561674, 0....","[0.09653562498312972, 0.06985888342346794, 0.0...","[0.07654920307129107, 0.04588245032601229, 0.0..."
2,Elastic Net,0.033444,0.001317,0.055751,0.026746,0.105524,0.052839,254.9202,"{'alpha': 9.973108826045255, 'copy_X': True, '...","[0.08847214244434784, 0.07839945934019509, 0.1...","[0.03198652928251123, 0.032189130781534146, 0....","[0.11724506861407157, 0.07023870950289694, 0.0...","[0.09367995606956717, 0.07080361652676895, 0.0..."
3,Random Forest,0.076490,0.002545,0.146966,0.028906,0.183533,0.031418,556.6586,"{'max_depth': 38, 'max_leaf_nodes': 21, 'min_s...","[0.19007065800038656, 0.20236809026495145, 0.1...","[0.07942972944919592, 0.0805806648357526, 0.07...","[0.10649519204948915, 0.12835186644190952, 0.1...","[0.17870582493116696, 0.19241889064618928, 0.1..."
4,K-Nearest Neighbors,0.000000,0.000000,0.148266,0.047779,0.186641,0.043100,1.0994,"{'algorithm': 'brute', 'leaf_size': 39, 'metri...","[0.18547693791215658, 0.22947100158692083, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.12798394574174418, 0.05815775988032873, 0.1...","[0.20924416332038512, 0.2362024093676182, 0.15..."


In [ ]:
# Check whether MAPE value of optimised regressors beat the best baseline value

hpo_results['HPO train MAPE'] = (hpo_results['HPO train MAPE'] * 100).round(2)
hpo_results['Train std'] = (hpo_results['Train std'] * 100).round(2)
hpo_results['HPO valid MAPE'] = (hpo_results['HPO valid MAPE'] * 100).round(2)
hpo_results['Valid std'] = (hpo_results['Valid std'] * 100).round(2)
hpo_results['HPO test MAPE'] = (hpo_results['HPO test MAPE'] * 100).round(2)
hpo_results['Test std'] = (hpo_results['Test std'] * 100).round(2)

# Add a column which says if the regressor beat the baseline
hpo_results['beats_bl'] = np.where(hpo_results.loc[:,'HPO test MAPE'] < best_baseline_score, 'yes', 'no')

# See the HPO results
hpo_results

,regressor_name,HPO train MAPE,Train std,HPO valid MAPE,Valid std,HPO test MAPE,Test std,runtime_hpo,best_params,FH_MAPEs,train_MAPE_per_split,cv_MAPE_per_split,test_MAPE_per_split,beats_bl
0,XGBoost,6.76,0.12,5.77,2.17,8.61,4.52,717.1597,"{'booster': 'gblinear', 'feature_selector': 'c...","[0.07883665244247692, 0.06773080034489458, 0.0...","[0.06743986537476192, 0.06881291584713467, 0.0...","[0.0703990174229799, 0.06667889811138855, 0.09...","[0.06114039102739344, 0.01854096454474325, 0.0...",yes
1,Ridge,4.29,0.05,5.55,2.44,9.87,5.60,92.9519,"{'alpha': 0.5438252426248501, 'fit_intercept':...","[0.08261525279811104, 0.06980042020881116, 0.1...","[0.04317532704468448, 0.042872883135561674, 0....","[0.09653562498312972, 0.06985888342346794, 0.0...","[0.07654920307129107, 0.04588245032601229, 0.0...",no
2,Elastic Net,3.34,0.13,5.58,2.67,10.55,5.28,254.9202,"{'alpha': 9.973108826045255, 'copy_X': True, '...","[0.08847214244434784, 0.07839945934019509, 0.1...","[0.03198652928251123, 0.032189130781534146, 0....","[0.11724506861407157, 0.07023870950289694, 0.0...","[0.09367995606956717, 0.07080361652676895, 0.0...",no
3,Random Forest,7.65,0.25,14.70,2.89,18.35,3.14,556.6586,"{'max_depth': 38, 'max_leaf_nodes': 21, 'min_s...","[0.19007065800038656, 0.20236809026495145, 0.1...","[0.07942972944919592, 0.0805806648357526, 0.07...","[0.10649519204948915, 0.12835186644190952, 0.1...","[0.17870582493116696, 0.19241889064618928, 0.1...",no
4,K-Nearest Neighbors,0.00,0.00,14.83,4.78,18.66,4.31,1.0994,"{'algorithm': 'brute', 'leaf_size': 39, 'metri...","[0.18547693791215658, 0.22947100158692083, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.12798394574174418, 0.05815775988032873, 0.1...","[0.20924416332038512, 0.2362024093676182, 0.15...",no


In [ ]:
bl_beaters = hpo_results[(hpo_results['beats_bl'] == 'yes')]
bl_beaters[bl_beaters['runtime_hpo'] == bl_beaters['runtime_hpo'].min()]

,regressor_name,HPO train MAPE,Train std,HPO valid MAPE,Valid std,HPO test MAPE,Test std,runtime_hpo,best_params,FH_MAPEs,train_MAPE_per_split,cv_MAPE_per_split,test_MAPE_per_split,beats_bl
0,XGBoost,6.76,0.12,5.77,2.17,8.61,4.52,717.1597,"{'booster': 'gblinear', 'feature_selector': 'c...","[0.07883665244247692, 0.06773080034489458, 0.0...","[0.06743986537476192, 0.06881291584713467, 0.0...","[0.0703990174229799, 0.06667889811138855, 0.09...","[0.06114039102739344, 0.01854096454474325, 0.0...",yes


### Final results

In [ ]:
# Merge dataframes on regressor_name
merged_df = pd.merge(hpo_results, reg_results_besttest[['Regressor',	'Mean Train MAPE',	'Mean Validation MAPE',	'Mean Test MAPE']], left_on='regressor_name', right_on='Regressor', how='left')

# Add a column indicating whether HPO test_acc beats the baseline regression_r2
merged_df['improved_after_hpo'] = np.where(merged_df['HPO test MAPE'] < merged_df['Mean Test MAPE'], 'yes', 'no')

columns_to_drop = ['runtime_hpo', 'best_params', 'Regressor', 'FH_MAPEs', 'train_MAPE_per_split', 'cv_MAPE_per_split', 'test_MAPE_per_split']
merged_df = merged_df.drop(columns=columns_to_drop)

# Display the final dataframe
merged_df

,regressor_name,HPO train MAPE,Train std,HPO valid MAPE,Valid std,HPO test MAPE,Test std,beats_bl,Mean Train MAPE,Mean Validation MAPE,Mean Test MAPE,improved_after_hpo
0,XGBoost,6.76,0.12,5.77,2.17,8.61,4.52,yes,0.00,17.24,18.38,yes
1,Ridge,4.29,0.05,5.55,2.44,9.87,5.60,no,5.50,5.84,8.96,no
2,Elastic Net,3.34,0.13,5.58,2.67,10.55,5.28,no,15.28,14.86,14.38,yes
3,Random Forest,7.65,0.25,14.70,2.89,18.35,3.14,no,7.19,15.39,19.05,yes
4,K-Nearest Neighbors,0.00,0.00,14.83,4.78,18.66,4.31,no,16.10,16.42,18.38,no


In [ ]:
baselines

,Unnamed: 0,sNaive valid,sNaive test,wNaive valid,wNaive test
0,Mean MAPE,0.1434,0.1721,0.148,0.1711
1,MAPE std,0.0633,0.0372,0.0653,0.0412
2,Mean RMSE,12002.2089,13204.6307,12630.405,12937.8088
3,Mean R2,-1.4037,-1.4762,-1.7852,-1.5385
4,FH MAPEs,***,"[0.1878773278197889, 0.18045408841835758, 0.17...",***,"[0.18873723128298953, 0.18269765563726237, 0.1..."


In [ ]:
columns_to_convert = ['sNaive valid', 'sNaive test', 'wNaive valid', 'wNaive test']
baselines.loc[:3, columns_to_convert] = baselines.loc[:3, columns_to_convert].astype(float)

In [ ]:
# Add sNaive and wNaive rows
snaive_row = {'regressor_name': 'sNaive', 'HPO train MAPE': '***', 'Train std': '***','HPO valid MAPE': round(baselines.iloc[0][1]*100,2), 'Valid std': round(baselines.iloc[1][1]*100,2), 'HPO test MAPE': round(baselines.iloc[0][2]*100,2),
              'Test std': round(baselines.iloc[1][2]*100,2), 'Mean Train MAPE': '***','Mean Validation MAPE': '***','Mean Test MAPE': '***','improved_after_hpo': '***'}
wnaive_row = {'regressor_name': 'wNaive', 'HPO train MAPE': '***', 'Train std': '***', 'HPO valid MAPE': round(baselines.iloc[0][3]*100,2), 'Valid std': round(baselines.iloc[1][3]*100,2), 'HPO test MAPE': round(baselines.iloc[0][4]*100,2),
              'Test std': round(baselines.iloc[1][4]*100,2), 'Mean Train MAPE': '***','Mean Validation MAPE': '***','Mean Test MAPE': '***','improved_after_hpo': '***'}
merged_df = pd.concat([pd.DataFrame([snaive_row, wnaive_row]), merged_df], ignore_index=True)

# Calculate 'formula*' for the added rows
merged_df.loc[merged_df['regressor_name'].isin(['sNaive', 'wNaive']), 'beats_bl'] = (
    merged_df['HPO test MAPE'] < best_baseline_score
).map({True: 'yes', False: 'no'})

# Insert 'beats_bl' column after 'HPO test MAPE'
merged_df.insert(7, 'beats_bl', merged_df.pop('beats_bl'))

merged_df

,regressor_name,HPO train MAPE,Train std,HPO valid MAPE,Valid std,HPO test MAPE,Test std,beats_bl,Mean Train MAPE,Mean Validation MAPE,Mean Test MAPE,improved_after_hpo
0,sNaive,***,***,14.34,6.33,17.21,3.72,no,***,***,***,***
1,wNaive,***,***,14.80,6.53,17.11,4.12,no,***,***,***,***
2,XGBoost,6.76,0.12,5.77,2.17,8.61,4.52,yes,0.0,17.24,18.38,yes
3,Ridge,4.29,0.05,5.55,2.44,9.87,5.60,no,5.5,5.84,8.96,no
4,Elastic Net,3.34,0.13,5.58,2.67,10.55,5.28,no,15.28,14.86,14.38,yes
5,Random Forest,7.65,0.25,14.70,2.89,18.35,3.14,no,7.19,15.39,19.05,yes
6,K-Nearest Neighbors,0.0,0.0,14.83,4.78,18.66,4.31,no,16.1,16.42,18.38,no


In [ ]:
hpo_results.to_csv('reg_hpo_results_ar_300_v4.csv', index=False)
merged_df.to_csv('reg_results_merged_ar_300_v4.csv', index=False)